In [26]:
import warnings
warnings.filterwarnings('ignore')  # 警告扰人，手动封存
import torch
from DataLoader_s2s import Exampledataset,collate_func
from model import S2sTransformer,seq_generation_loss
from AF_LSTM import AF_LSTM
from utils.vocabulary import Vocab
import logging
from torch.utils.data import DataLoader
from tqdm import tqdm
import textstat
from utils.Word2Vec_emb import Word_Embedding
from rouge import Rouge
import json, os, math
import numpy as np
import torch.nn.functional as F
import heapq
from torch.nn.utils.rnn import pad_sequence
from nltk.translate import bleu_score

In [27]:
def jsonloader(filename):
    # 将数据加载到一个列表中
    file = open(filename, 'r', encoding='utf-8')
    entity_list=[]#实体
    news_list = []#新闻
    date_list = []#日期
    vnames_list = []#新闻所含实体列表
    label_list=[]#评论
    title_list=[]#标题
    label_score_list=[]#评论情感分数
    for line in file.readlines():
        pop_dict = json.loads(line)
        entity=pop_dict['entity']
        date = pop_dict['date']
        news = pop_dict['news']
        vnames = pop_dict['v_names']
        label=pop_dict['label']
        title=pop_dict['title']
        #label_score=pop_dict['label_score']

        news_list.append(news)
        date_list.append(date)
        entity_list.append(entity)
        vnames_list.append(vnames)
        label_list.append(label)
        title_list.append(title)
        #label_score_list.append(label_score)
    return entity_list,news_list, date_list, vnames_list,label_list,label_score_list,title_list

In [28]:
def searchcomment(label_list):
    cur_label_list=[]
    for label in label_list:
        cur_label_list.append(label[0])
    return cur_label_list

In [29]:
def evalreadlibility(text):
    #text=[word,word-.....]为word的列表
    test_data = ' '.join(text)
    flesch_reading_ease=textstat.flesch_reading_ease(test_data)
    smog_index=textstat.smog_index(test_data)
    flesch_kincaid_grade=textstat.flesch_kincaid_grade(test_data)
    coleman_liau_index=textstat.coleman_liau_index(test_data)
    automated_readability_index=textstat.automated_readability_index(test_data)
    dale_chall_readability_score=textstat.dale_chall_readability_score(test_data)
    difficult_words=textstat.difficult_words(test_data)
    linsear_write_formula=textstat.linsear_write_formula(test_data)
    gunning_fog=textstat.gunning_fog(test_data)
    text_standard=textstat.text_standard(test_data,float_output=True)

    return flesch_reading_ease,smog_index,flesch_kincaid_grade,coleman_liau_index,automated_readability_index,dale_chall_readability_score,difficult_words,linsear_write_formula,gunning_fog,text_standard

In [30]:
def Rouge_score(a,b):
    a = ' '.join(a)
    b = ' '.join(b)
    rouge = Rouge()
    rouge_score = rouge.get_scores(a, b)
    rouge_1=rouge_score[0]["rouge-1"]['r']
    rouge_2=rouge_score[0]["rouge-2"]['r']
    rouge_L=rouge_score[0]["rouge-l"]['r']
    return rouge_1,rouge_2,rouge_L
def Blue_score(a, b):

    b = [b]
    smooth = bleu_score.SmoothingFunction()
    return bleu_score.sentence_bleu(b,a,smoothing_function=smooth.method2)

In [31]:
def greedysearch(decoder, max_len, query, vocab):
    """
    a greedy search implementation about seq2seq transformer
    :param decoder:
    :param max_len: max length of result
    :param query: input of Encoder
    :param vocab: vocab
    :return: list of index
    """
    cnt=0
    res=[]
    src_ids=[vocab.word2id(word) for word in query]
    src = torch.tensor(src_ids).unsqueeze(0)
    tgt_ids=[vocab.word2id('[START]')]
    while cnt<max_len:
        tgt=torch.tensor(tgt_ids).unsqueeze(0)
        out=decoder.forward(src,tgt)
        cur=out[-1,0,:].argmax().item()
        if vocab.id2word(cur)=='[STOP]':
            break
        tgt_ids.append(cur)
        res.append(vocab.id2word(cur))
        cnt+=1
    return ''.join(res),res

In [32]:
class beamheap:
    def __init__(self,vocab,model,Beamsize,backaerfa,foraerfa):
        self.vocab=vocab
        #self.model=transformer_base(self.vocab,model_params['embed_dim'],model_params['nheads'],pretrained_weight)
        self.model=model
        self.Beamsize=Beamsize
        self.backaerfa=backaerfa#长度惩罚因子
        self.foraerfa=foraerfa
        
    def back_decode(self,batch,tgt_ids,devices):
        self.model.eval()
        #attention_mask=starattentionmask(src.size(1))
        tgt=torch.tensor(tgt_ids).unsqueeze(0)
        tgt = tgt.to(device)
        _,back_out=self.model(src_ids = batch['src_ids'],
                                   back_tgt_ids = tgt,
                                   for_tgt_ids = tgt,
                                   src_pad_mask = batch['src_pad_mask'],
                                   new_list = batch['new_ids'],
                                   new_mask = batch['new_ids_mask'],
                                   entity = batch['entity'],
                                   gpunum = int(devices))
        #print(back_out.shape)
        return back_out # 输出维度维度是V*1 ，代表每个词出现的概率
    
    def for_decode(self,batch,tgt_ids,devices):
        self.model.eval()
        #attention_mask=starattentionmask(src.size(1))
        tgt=torch.tensor(tgt_ids).unsqueeze(0)
        tgt = tgt.to(device)
        
        for_out,_=self.model(src_ids = batch['src_ids'],
                                   back_tgt_ids = tgt,
                                   for_tgt_ids = tgt,
                                   src_pad_mask = batch['src_pad_mask'],
                                   new_list = batch['new_ids'],
                                   new_mask = batch['new_ids_mask'],
                                   entity = batch['entity'],
                                   gpunum = int(devices))
        return for_out # 输出维度维度是V*1 ，代表每个词出现的概率

    def fun(self,que,x,dir=None):# 这块维持一个大小为k（Beam search的宽度）的小顶堆，使得复杂度降到log(k)
        k=self.Beamsize
        if dir == "back":
            aerfa=self.backaerfa
        elif dir == "for":
            aerfa=self.foraerfa
        else:
            raise RuntimeError("the direction of decode is unlogic")
        que.append(x)
        def second(key):# 以序列出现的概率为依据进行排序
            p=key[1]
            L=len(key[0])
            M=math.log(p)/(math.pow( L,aerfa))
            return M
        que.sort(key=second)
        if len(que)>k:
            que=que[-k:]
        return que

In [33]:
def updatafreq(beams,beamsfreq):
    for dic in beams:
        word=dic
        beamsfreq[word]+=1

In [34]:
class beam_search_decoder:
    def __init__(self,beamheap,maxlen,numda,minlen):
        self.back_decode = beamheap.back_decode#前向decode和后向decode
        self.for_decode = beamheap.for_decode
        self.topk = beamheap.fun# 这是用于更新当前保留较大的K个备选项的函数
        self.vocab = beamheap.vocab
        self.maxlen = maxlen
        self.beamsize = beamheap.Beamsize
        self.numda = numda
        self.minlen = minlen
    def back_forward(self,batch,entity,devices):# 模型输入为初始向量，一般是编码器的输出
        beams = [([entity],1.0)]# 首先把初始向量填入beam中 第一值是输出的序列列表，第二值是该序列出现的概率
        beamsfreq=[1 for _ in range(self.vocab.size())]
        for itername in range(self.maxlen):# 自回归式迭代生成输出序列 最大输出序列长度为max_len
            que = [] # 临时beam缓存
            for x,score in beams:# 遍历Beam中所有备选项
                if x[-1]==2:#BOS id = 2 如果已经输出了开始字符 则该序列直接用于更新，不再进行解码
                    que = self.topk(que,(x,score),'back')
                else:
                    output = self.back_decode(batch,x,devices) # 以Beam中已生成的序列为输入，生成下一token的概率分布
                    output = output[-1,0,:]
                    output = F.softmax(output)
                    output = output.cpu()
                    if self.numda!=0:
                        tgt_freq=torch.FloatTensor(beamsfreq)
                        tgt_freq=torch.pow(tgt_freq, self.numda)
                        tgt_f_vocab=1/tgt_freq
                        output=output*tgt_f_vocab
                    output=output.tolist()
                    beammax=heapq.nlargest(self.beamsize,range(len(output)),output.__getitem__)
                    # if itername<self.minlen:

                    updatafreq(beammax,beamsfreq)
                    for wid in beammax:# 遍历输出选中的beamsize个词
                        o_score=output[wid]
                        que = self.topk(que,(x+[wid],score*o_score),'back')# 假设改词为输出的词，那么可以得到一个新的序列以及该序列出现的概率
            beams = que # 更新Beam
        return beams[-1][0]
    def for_forward(self,src,back_seq,devices):
        beams = [(back_seq,1.0)]# 首先把初始向量填入beam中 第一值是输出的序列列表，第二值是该序列出现的概率
        beamsfreq=[1 for _ in range(self.vocab.size())]
        for itername in range(self.maxlen):# 自回归式迭代生成输出序列 最大输出序列长度为max_len
            que = [] # 临时beam缓存
            for x,score in beams:# 遍历Beam中所有备选项
                if x[-1]==3:#EOS id = 3 如果已经输出了结束字符 则该序列直接用于更新，不再进行解码
                    que = self.topk(que,(x,score),'for')
                else:
                    output = self.for_decode(src,x,devices) # 以Beam中已生成的序列为输入，生成下一token的概率分布
                    output = output[-1,0,:]
                    output = F.softmax(output)
                    output = output.cpu()
                    if self.numda!=0:
                        tgt_freq=torch.FloatTensor(beamsfreq)
                        tgt_freq=torch.pow(tgt_freq, self.numda)
                        tgt_f_vocab=1/tgt_freq
                        output=output*tgt_f_vocab
                    output=output.tolist()
                    beammax=heapq.nlargest(self.beamsize,range(len(output)),output.__getitem__)
                    updatafreq(beammax,beamsfreq)
                    for wid in beammax:# 遍历输出选中的beamsize个词
                        o_score=output[wid]
                        que = self.topk(que,(x+[wid],score*o_score),'for')# 假设改词为输出的词，那么可以得到一个新的序列以及该序列出现的概率
            beams = que # 更新Beam
        return beams[-1][0]

In [44]:
datasetname = "entertainment"
args={
    # 数据文件路径
    'test_path':"./data/generate_data/"+ datasetname + "_data/"+ datasetname + "_test_49500_2gram.json",
    # vocab路径
    'vocab_path':"./data/generate_data/"+ datasetname + "_data/"+ datasetname + "_vocab_49500_2gram.txt",
    # senti_vocab路径
    'senti_vocab_path':"./data/sentiment_data/"+ datasetname + "_data/"+ datasetname + "_sentiment_vocab.txt",
    # 词典最大长度
    'vocab_len':110000,
    'senti_vocab_len':150000,
    # 模型保存路径
    'checkpoint_path':'./ckpt/',
    'history_path':'./history/',
    # 情感分析模型路径
    'result_model_path':"./"+datasetname+"_ckpt/GModel_steps_250000.pkl",
    # 日志路径
    'log_path':'./log/',
    # 预训练词embedding模型路径
    'embedding_path':'./Word2Vec/word_embedding', 
    'word_emb_dim':128, # default: 128
    'nheads_transformer':4, # embed_dim % nheads_transformer == 0
    # 是否使用之前的checkpoint ，0则为不使用
    'resume':0,
    'model_save_name':'transback_00_2gram_itf_sport',
    # only test
    'model_resume_name':'',
    # 训练batch参数
    'batch_size':8,
    'end_epoch':100,
    'check_steps':1000,
    # 模型保存间隔步数
    'save_steps':2000,
    'lr':1e-4,
    # 输出loss间隔步数
    'loss_check':300,
    # only test
    'version_info':'use pretrained embed , encode_layers=6 model.train() revise',
    'GPU_ids':'7',
    # 学习率递减
    'lr_descent':False,
    # 最小学习率
    'minlr':5e-5,
    # 加载预训练好的embedding数据路径
    'pretrain_path':"data/generate_data/"+ datasetname + "_data/pretrained_weight_"+ datasetname + "_vocab_49500_2gram.npy",
    # 加载预训练好的senti_embedding数据路径
    'senti_pretrain_path':"data/sentiment_data/"+ datasetname + "_data/pretrained_weight_"+ datasetname + "_sentiment_vocab.npy",
    # 词频字典路径
    'freq_path': "data/generate_data/"+ datasetname + "_data/"+ datasetname + "_49500_2-gram_labelFre.json",
    # 超参
    'numda':0.4,
    'gama':0,
    'sparse_attention':False,
    'devices':'0'
}

In [45]:
test_path = args['test_path']
vocab_path = args['vocab_path']
senti_vocab_path = args['senti_vocab_path']
vocab_len = args['vocab_len']
senti_vocab_len = args['senti_vocab_len']
resume = args['resume']
checkpoint_path = args['checkpoint_path']
history_path    = args['history_path']
log_path = args['log_path']
model_name = args['model_save_name']
model_resume_name = args['model_resume_name']
batch_size = args['batch_size']
end_epoch  = args['end_epoch']
lr = args['lr']
loss_check_freq = args['loss_check']
check_steps= args['check_steps']
save_steps = args['save_steps']
#os.environ['CUDA_VISIBLE_DEVICES'] = args['GPU_ids']
embedding_path= args['embedding_path']
word_emb_dim = args['word_emb_dim']
nheads = args['nheads_transformer']
minlr= args['minlr']
lr_descent= args['lr_descent']
pretrain_path= args['pretrain_path']
senti_pretrain_path = args['senti_pretrain_path']
GPU_ids=args['GPU_ids']
numda=args['numda']
gama=args['gama']
freq_path=args['freq_path']
sparse_attention=args['sparse_attention']
devices = args['devices']
device = torch.device("cuda:"+devices if torch.cuda.is_available() else "cpu")

In [46]:
print(device)

cuda:0


In [47]:
#加载生成词典
vocab_list=[]
for line in open(vocab_path, "r",encoding = 'utf-8'):
    vocab_list.append(line[:-1])
print("[INFO] vocab_list读取成功！")
print("[INFO] vocab_size:" , len(vocab_list))
# 创建vocab类
vocab = Vocab(vocab_list, vocab_len)

#加载情感模型词典
vocab_list2=[]
for line in open(senti_vocab_path, "r",encoding = 'utf-8'):
    vocab_list2.append(line[:-1])
print("[INFO] senti_vocab_list读取成功！")
print("[INFO] senti_vocab_size:" , len(vocab_list2))
# 创建sentivocab类
senti_vocab = Vocab(vocab_list2, senti_vocab_len)

[INFO] vocab_list读取成功！
[INFO] vocab_size: 70678
[INFO] Finished constructing vocabulary of %i total words. Last word added: %s 70682 芝迷们
[INFO] senti_vocab_list读取成功！
[INFO] senti_vocab_size: 72765
[INFO] Finished constructing vocabulary of %i total words. Last word added: %s 72769 报国


In [48]:
# 加载 生成预训练embed
if not os.path.exists(pretrain_path):
    print("无pretrain,加载中")
    embed_loader = Word_Embedding(embedding_path, vocab)
    vectors = embed_loader.load_my_vecs()
    pretrained_weight = embed_loader.add_unknown_words_by_uniform(vectors, word_emb_dim)
    np.save(pretrain_path,pretrained_weight)
    print("save完成")
pretrained_weight = np.load(pretrain_path)
pretrained_weight = torch.from_numpy(pretrained_weight).to(device)

# 加载 情感预训练embed
if not os.path.exists(senti_pretrain_path):
    print("无pretrain,加载中")
    embed_loader = Word_Embedding(embedding_path, senti_vocab)
    vectors = embed_loader.load_my_vecs()
    senti_pretrained_weight = embed_loader.add_unknown_words_by_uniform(vectors, word_emb_dim)
    np.save(senti_pretrain_path,senti_pretrained_weight)
    print("save完成")
senti_pretrained_weight = np.load(senti_pretrain_path)
senti_pretrained_weight = torch.from_numpy(senti_pretrained_weight).to(device)

In [49]:
AFLSTM_model = AF_LSTM(senti_vocab, ifNorm=True, num_layers=1, pretrained_embeddings=senti_pretrained_weight)
model = S2sTransformer(vocab, senti_model = AFLSTM_model, word_emb_dim = word_emb_dim, nhead = nheads, pretrained_weight = pretrained_weight)
model.to(device)

Model : AF_LSTM is ready to run


S2sTransformer(
  (embedding): Embedding(70682, 128)
  (pos_encoder): LearnedPositionEncoding(
    500, 128
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): Linear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): Linear(in_features=128, out_features=128, bias=True)
        )
        (linear

In [50]:
model.load_state_dict(torch.load(args['result_model_path']))

<All keys matched successfully>

In [51]:
print(test_path)

./data/generate_data/entertainment_data/entertainment_test_49500_2gram.json


In [52]:
testset = Exampledataset(test_path, vocab,senti_vocab, "test",freq_path)

In [53]:
vocab.id2word(34132)

'著名主持人'

In [54]:
vocab.word2id('梅西')

40551

In [55]:
len(testset)

500

In [56]:
print(testset[0])

{'title_token': [43611, 58669, 62162, 63710, 44907, 8588, 70321, 6947, 27382, 40485, 39146, 16582, 25217, 6604, 53245, 58401, 68465, 35667], 'new_token': [24660, 17579, 12157, 58810, 49667, 17990, 64562, 23593, 64352, 71124, 70254, 26094, 18497, 31017, 9128, 29801, 60147, 795, 25959, 65143, 5633, 26804, 25273, 13560, 47332, 61149, 11056, 5225, 38327, 3493, 49667, 17990, 21445, 20149, 61754, 18497, 43257, 40442, 9489, 64578, 10950, 61656, 43333, 17846, 38787, 62786, 62149, 34792, 4822, 9489, 25388, 72653, 18652, 58810, 28564, 32730, 36259, 61671, 70932, 51628, 7238, 61285, 58810, 19413, 21276, 17846, 58810, 15116, 51931, 72653, 66170, 32091, 48281, 36103, 65930, 58810, 66724, 50270, 58810, 43333, 49340, 12709, 3927, 64493, 52725, 6524, 45812, 58810, 66724, 21276, 43333, 66724, 28300, 45812, 33659, 19372, 37755, 18497, 4791, 9643, 66724, 61754, 21799, 62304, 35724, 37609, 8382, 67224, 60715, 12676, 13435, 59926, 59926, 27600, 11912], 'entity': [58810], 'for_comment_token': [43611, 65378,

In [ ]:
test_loader = DataLoader(testset, batch_size=1,num_workers=4, shuffle=False,collate_fn=collate_func,drop_last=True)

In [23]:
Beamsize=6
backaerfa=1.7
foraerfa=1.38
numda=0.6
minlen=5
comment_maxlen = 20
beamsearchheap=beamheap(vocab,model,Beamsize,backaerfa,foraerfa)
beam_search_decode=beam_search_decoder(beamsearchheap,comment_maxlen,numda,minlen)

In [24]:
flesch_reading_ease_score=0
smog_index_score=0
flesch_kincaid_grade_score=0
coleman_liau_index_score=0
automated_readability_index_score=0
dale_chall_readability_score_score=0
difficult_words_score=0
linsear_write_formula_score=0
gunning_fog_score=0
text_standard_score=0
rouge_1_list,rouge_2_list,rouge_L_list=0,0,0
bleu_list=0

In [ ]:
for index, batch in enumerate(test_loader):
    src_batch, \
    back_tgt_batch, \
    for_tgt_batch, \
    src_pad_mask, \
    back_tgt_pad_mask, \
    for_tgt_pad_mask, \
    back_tgt_att_mask, \
    for_tgt_att_mask, \
    src_mask_batch, \
    back_tgt_freq_batch, \
    for_tgt_freq_batch , \
    back_tgt_pos_batch, \
    for_tgt_pos_batch , \
    new_ids_batch , \
    new_ids_mask_batch , \
    entity_batch = \
        batch['src_ids'], \
        batch['back_tgt_ids'], \
        batch['for_tgt_ids'], \
        batch['src_pad_mask'], \
        batch['back_tgt_pad_mask'], \
        batch['for_tgt_pad_mask'], \
        batch['back_tgt_mask'], \
        batch['for_tgt_mask'], \
        batch['src_mask'],\
        batch['back_tgt_freq'], \
        batch['for_tgt_freq'], \
        batch['back_tgt_pos'], \
        batch['for_tgt_pos'], \
        batch['new_ids'], \
        batch['new_ids_mask'], \
        batch['entity']
    
    batch = {k: v.to(device)for k,v in batch.items()}
    
    entity = batch['entity'].squeeze().item()
    entity = vocab.word2id(senti_vocab.id2word(entity))
    
    print(vocab.id2word(entity))
     
    # 先生成前向序列
    prefix_comment=beam_search_decode.back_forward(batch,entity,devices)
    # 逆序
    prefix_comment=prefix_comment[::-1]
    # labellist = []
    # for word in prefix_comment:
    #     labellist.append(vocab.id2word(word))
    # print(labellist)
    # 生成后向序列
    complete_comment=beam_search_decode.for_forward(batch,prefix_comment,devices)
    if complete_comment[-1]!=3:
        complete_comment.append(3)
    complete_comment=[vocab.id2word(cur) for cur in complete_comment]

    print('comment'+str(index)+' predict:',''.join(complete_comment))

    title = testset[index]['title_token']
    titlelist = []
    for word in title:
        titlelist.append(vocab.id2word(word))
    title = ''.join(titlelist)
    print(title)
    
    back_label = testset[index]['back_comment_token']
    for_label = testset[index]['for_comment_token']
    back_label = back_label[::-1]
    label = back_label[:-1]+for_label
    labellist = []
    for word in label:
        labellist.append(vocab.id2word(word))
        
    label = ''.join(labellist)
    print(label)
    # label.insert(0,"[START]")
    # label.append("[STOP]")
    # predict.insert(0,"[START]")
    # predict.append("[STOP]")

    #print('labels',index,''.join(label))
    flesch_reading_ease,\
    smog_index,\
    flesch_kincaid_grade,\
    coleman_liau_index,\
    automated_readability_index,\
    dale_chall_readability_score,\
    difficult_words,\
    linsear_write_formula,\
    gunning_fog,\
    text_standard=evalreadlibility(complete_comment)
    rouge_1,rouge_2,rouge_L=Rouge_score(complete_comment,label)
    bleu=Blue_score(complete_comment,label)
    bleu_list+=bleu
    rouge_1_list+=rouge_1
    rouge_2_list+=rouge_2
    rouge_L_list+=rouge_L
    flesch_reading_ease_score+=flesch_reading_ease
    smog_index_score+=smog_index
    flesch_kincaid_grade_score+=flesch_kincaid_grade
    coleman_liau_index_score+=coleman_liau_index
    automated_readability_index_score+=automated_readability_index
    dale_chall_readability_score_score+=dale_chall_readability_score
    difficult_words_score+=difficult_words
    linsear_write_formula_score+=linsear_write_formula
    gunning_fog_score+=gunning_fog
    text_standard_score+=text_standard
n_iter=len(news_list)
print("rouge_1:",rouge_1_list/n_iter)
print("rouge_2:",rouge_2_list/n_iter)
print("rouge_L:",rouge_L_list/n_iter)
print("bleu:",bleu_list/n_iter)
print("flesch_reading_ease_score:",flesch_reading_ease_score/n_iter)
print("smog_score:",smog_index_score/n_iter)
print("flesch_kincaid_grade_score:",flesch_kincaid_grade_score/n_iter)
print("coleman_liau_index_score:",coleman_liau_index_score/n_iter)
print("automated_readability_score:",automated_readability_index_score/n_iter)
print("dale_chall_readability_score:",dale_chall_readability_score_score/n_iter)
print("difficult_words_score:",difficult_words_score/n_iter)
print("gunning_fog_score:",gunning_fog_score/n_iter)
print("linsear_write_formula_score:",linsear_write_formula_score/n_iter)
print("text_standard_score:",text_standard_score/n_iter)

李小冉
comment0 predict: 第三部第三部第三部第三部第三部第三部第三部第三部第三部第三部第三部第三部第三部第三部第三部第三部第三部第三部第三部第三部李小冉第三部第三部第三部第三部第三部第三部第三部第三部第三部第三部第三部第三部第三部第三部第三部第三部第三部第三部第三部第三部[STOP]
李小冉她为前男友怀孕没结婚后嫁给苦守16年的男闺蜜如今被宠成公主
[START]李小冉漂亮但说真话一张苦命脸辨识度[STOP]
李菁
